In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from Metric import get_uplift_model_aucc
from model import slearner_criteo_uplift
from utils import * 
from Lagrangian_duality_gradient_estimator import * 


In [2]:
df_criteo = pd.read_csv('data/criteo-uplift-v2.1.csv', sep=',')

random_state = 20220720
df_criteo = df_criteo.sample(frac=1.0, random_state=random_state).reset_index(drop=True)

In [3]:
df_criteo

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,12.616365,10.059654,8.859005,4.679882,10.280525,4.115453,0.294443,4.833815,3.910792,13.190056,5.300375,-0.168679,1,0,0,0
1,25.527349,10.059654,8.214383,4.679882,10.280525,4.115453,-2.411115,4.833815,3.971858,13.190056,5.300375,-0.168679,0,0,0,0
2,24.753301,10.059654,8.214383,4.679882,10.280525,4.115453,-1.288207,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0,0
3,24.544662,10.059654,8.214383,4.679882,10.280525,4.115453,-3.282109,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0,0
4,26.311796,10.059654,8.214383,4.679882,10.280525,4.115453,-1.288207,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13979587,12.616365,10.059654,8.868975,4.679882,10.280525,4.115453,0.294443,4.833815,3.876391,13.190056,5.300375,-0.168679,1,0,0,0
13979588,14.974642,10.059654,8.214383,3.907662,10.280525,4.115453,-8.493011,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0,0
13979589,14.461671,10.679513,8.214383,0.604065,10.280525,2.230907,-11.858523,7.900615,3.971858,13.190056,5.300375,-0.168679,1,0,0,0
13979590,13.717872,10.059654,8.280712,2.587543,10.280525,4.115453,-17.661325,4.833815,3.800545,31.144700,5.300375,-0.168679,1,0,0,0


In [4]:
T = df_criteo['treatment'].values.reshape(-1, 1)
unique, counts = np.unique(T, return_counts=True)
weights = counts / len(T)

train_dl, X_val, T_val, Y_visit_val, Y_visit_float_val, Y_conv_val, Y_conv_float_val, X_test, T_test, Y_visit_test, Y_visit_float_test, Y_conv_test, Y_conv_float_test = get_data(df_criteo, weights, unique, 300000, 30) 
weights = torch.tensor(weights).cuda()
X_val, T_val, Y_visit_val, Y_visit_float_val, Y_conv_val, Y_conv_float_val = X_val.cuda(), T_val.cuda(), Y_visit_val.cuda(), Y_visit_float_val.cuda(), Y_conv_val.cuda(), Y_conv_float_val.cuda()

In [5]:
with torch.no_grad():

    model_ts_best = slearner_criteo_uplift(12, 4)
    model_ts_best.load_state_dict(torch.load('Model/model_before_finetune.pkl'))
    T1 = torch.ones_like(T_test)
    T0 = torch.zeros_like(T_test)
    y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_ts_best(X_test, T0, T1)
    conv_pred = y_conv_pred_T1 - y_conv_pred_T0
    visit_pred = y_visit_pred_T1 - y_visit_pred_T0
    conv_pred = conv_pred.numpy()
    visit_pred =  visit_pred.numpy()
    roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
    test_aucc_ts = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
    print(f'test_aucc:{test_aucc_ts[0]}')

test_aucc:0.7576427022361585


In [6]:
#Improved Finite-Difference Strategy
lambda_1 = 0.1
max_val_aucc = 0
num_epoch = 10
h = 0.3
model_ifd = slearner_criteo_uplift(12, 4).cuda()
model_ifd.load_state_dict(torch.load('Model/model_before_finetune.pkl'))
optimizer = optim.Adam(model_ifd.parameters(), lr=1e-5, weight_decay=1e-5)
for epoch in range(num_epoch):
    for data in train_dl:
        optimizer.zero_grad()
        x, t, y_visit, y_visit_float, y_conv, y_conv_float = data
        x, t, y_visit, y_visit_float, y_conv, y_conv_float = x.cuda(), t.cuda(), y_visit.cuda(), y_visit_float.cuda(), y_conv.cuda(), y_conv_float.cuda()
        row_idx = torch.arange(x.shape[0])
        T1 = torch.ones_like(t).cuda()
        T0 = torch.zeros_like(t).cuda()
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_ifd(x, T0, T1)
        r_hat = torch.cat([y_conv_pred_T0, y_conv_pred_T1], dim=1)
        c_hat = torch.cat([y_visit_pred_T0, y_visit_pred_T1], dim=1)
        idx_T0 = torch.where(t==0)[0]
        idx_T1 = torch.where(t==1)[0]
        
        
        dQ_dr1, dQ_dc1 = improved_finite_difference(t=t, yr=y_conv_float, yc=y_visit_float, r_hat=r_hat, c_hat=c_hat, lambda_=lambda_1, weights=weights, h=h, clip='fix')

        dQ_dr1, dQ_dc1 = dQ_dr1.detach(), dQ_dc1.detach()

        loss_dfl = -torch.matmul(r_hat.t(), dQ_dr1).trace()

        loss = loss_dfl
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        model_ifd = model_ifd.cpu()
        T1 = torch.ones_like(T_test)
        T0 = torch.zeros_like(T_test)
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_ifd(X_test, T0, T1)
        conv_pred = y_conv_pred_T1 - y_conv_pred_T0
        visit_pred = y_visit_pred_T1 - y_visit_pred_T0
        conv_pred = conv_pred.numpy()
        visit_pred =  visit_pred.numpy()
        roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
        test_aucc = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
        model_ifd = model_ifd.cuda()

    with torch.no_grad():
        T1 = torch.ones_like(T_val).cuda()
        T0 = torch.zeros_like(T_val).cuda()
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_ifd(X_val, T0, T1)
        conv_pred = y_conv_pred_T1 - y_conv_pred_T0
        visit_pred = y_visit_pred_T1 - y_visit_pred_T0
        conv_pred = conv_pred.cpu().numpy()
        visit_pred =  visit_pred.cpu().numpy()
        roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
        val_aucc = get_uplift_model_aucc(t=(T_val.cpu().numpy() > 0.5).flatten(), yr=Y_conv_val.cpu().numpy().flatten(), yc=Y_visit_val.cpu().numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
        if val_aucc[0] > max_val_aucc:
            max_val_aucc = val_aucc[0]
            torch.save(model_ifd.state_dict(), 'Model/model_ifd.pkl')
    
    print(f'------epoch: {epoch+1}/{num_epoch}------val_aucc:{val_aucc[0]}------test_aucc:{test_aucc[0]}------')
    


with torch.no_grad():
    model_ifd_best = slearner_criteo_uplift(12, 4)
    model_ifd_best.load_state_dict(torch.load('Model/model_ifd.pkl'))
    T1 = torch.ones_like(T_test)
    T0 = torch.zeros_like(T_test)
    y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_ifd_best(X_test, T0, T1)
    conv_pred = y_conv_pred_T1 - y_conv_pred_T0
    visit_pred = y_visit_pred_T1 - y_visit_pred_T0
    conv_pred = conv_pred.numpy()
    visit_pred =  visit_pred.numpy()
    roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
    test_aucc_ifd = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
    print(f'------ifd finetune finished!------val_aucc:{max_val_aucc}------test_aucc:{test_aucc_ifd[0]}')


------epoch: 1/10------val_aucc:0.7190463173960582------test_aucc:0.7712565579734362------
------epoch: 2/10------val_aucc:0.716319472628475------test_aucc:0.7812341761551967------
------epoch: 3/10------val_aucc:0.7167091869075032------test_aucc:0.7859922524091434------
------epoch: 4/10------val_aucc:0.7218673560607439------test_aucc:0.7883989543843595------
------epoch: 5/10------val_aucc:0.723380065219531------test_aucc:0.7859323088843684------
------epoch: 6/10------val_aucc:0.7287265969330889------test_aucc:0.7870811410225322------
------epoch: 7/10------val_aucc:0.7306692934497881------test_aucc:0.7867055580705133------
------epoch: 8/10------val_aucc:0.726489715710019------test_aucc:0.7885779061475281------
------epoch: 9/10------val_aucc:0.7295609281811093------test_aucc:0.7875434116891156------
------epoch: 10/10------val_aucc:0.7268940863927515------test_aucc:0.787456024835904------
------ifd finetune finished!------val_aucc:0.7306692934497881------test_aucc:0.78670555807051

In [7]:
#Policy Learning Loss
lambda_1 = 0.1
max_val_aucc = 0
aucc_list = []
model_pl = slearner_criteo_uplift(12, 4).cuda()
model_pl.load_state_dict(torch.load('Model/model_before_finetune.pkl'))
optimizer = optim.Adam(model_pl.parameters(), lr=1e-5, weight_decay=1e-5)
for epoch in range(num_epoch):
    count = 0
    for data in train_dl:
        optimizer.zero_grad()
        x, t, y_visit, y_visit_float, y_conv, y_conv_float = data
        x, t, y_visit, y_visit_float, y_conv, y_conv_float = x.cuda(), t.cuda(), y_visit.cuda(), y_visit_float.cuda(), y_conv.cuda(), y_conv_float.cuda()
        row_idx = torch.arange(x.shape[0])
        
        T1 = torch.ones_like(t).cuda()
        T0 = torch.zeros_like(t).cuda()
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_pl(x, T0, T1)
        t = t.squeeze()
        r = torch.cat([y_conv_pred_T0, y_conv_pred_T1], dim=1)
        c = torch.cat([y_visit_pred_T0, y_visit_pred_T1], dim=1)
        idx_T0 = torch.where(t==0)[0]
        idx_T1 = torch.where(t==1)[0]
        
        
        a1 = nn.functional.softmax(r-lambda_1*c, dim=1)

        loss_dfl = - torch.mean((y_conv - lambda_1*y_visit).squeeze().detach() * a1[row_idx, t.long()])

        loss = loss_dfl
        loss.backward()
        optimizer.step()
        count += 1

    with torch.no_grad():
        model_pl = model_pl.cpu()
        T1 = torch.ones_like(T_test)
        T0 = torch.zeros_like(T_test)
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_pl(X_test, T0, T1)
        conv_pred = y_conv_pred_T1 - y_conv_pred_T0
        visit_pred = y_visit_pred_T1 - y_visit_pred_T0
        conv_pred = conv_pred.numpy()
        visit_pred =  visit_pred.numpy()
        roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
        test_aucc = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
        model_pl = model_pl.cuda()

    with torch.no_grad():
        T1 = torch.ones_like(T_val).cuda()
        T0 = torch.zeros_like(T_val).cuda()
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_pl(X_val, T0, T1)
        conv_pred = y_conv_pred_T1 - y_conv_pred_T0
        visit_pred = y_visit_pred_T1 - y_visit_pred_T0
        conv_pred = conv_pred.cpu().numpy()
        visit_pred =  visit_pred.cpu().numpy()
        roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
        val_aucc = get_uplift_model_aucc(t=(T_val.cpu().numpy() > 0.5).flatten(), yr=Y_conv_val.cpu().numpy().flatten(), yc=Y_visit_val.cpu().numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
        if val_aucc[0] > max_val_aucc:
            max_val_aucc = val_aucc[0]
            torch.save(model_pl.state_dict(), 'Model/model_pl.pkl')

    print(f'------epoch: {epoch+1}/{num_epoch}------val_aucc:{val_aucc[0]}------test_aucc:{test_aucc[0]}------')



with torch.no_grad():
    model_pl_best = slearner_criteo_uplift(12, 4)
    model_pl_best.load_state_dict(torch.load('Model/model_pl.pkl'))
    T1 = torch.ones_like(T_test)
    T0 = torch.zeros_like(T_test)
    y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_pl_best(X_test, T0, T1)
    idx_T0 = torch.where(T_test==0)[0]
    idx_T1 = torch.where(T_test==1)[0]
    conv_pred = y_conv_pred_T1 - y_conv_pred_T0
    visit_pred = y_visit_pred_T1 - y_visit_pred_T0
    conv_pred = conv_pred.numpy()
    visit_pred =  visit_pred.numpy()
    roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
    test_aucc_pl = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
    print(f'------pl finetune finished!------val_aucc:{max_val_aucc}------test_aucc:{test_aucc_pl[0]}')


------epoch: 1/10------val_aucc:0.7168119417098112------test_aucc:0.7672649371999091------
------epoch: 2/10------val_aucc:0.7284346157656674------test_aucc:0.7711795785830728------
------epoch: 3/10------val_aucc:0.7444387639114133------test_aucc:0.7731270850803473------
------epoch: 4/10------val_aucc:0.7406242648678697------test_aucc:0.7810324514832432------
------epoch: 5/10------val_aucc:0.7380698593422175------test_aucc:0.7784797118918335------
------epoch: 6/10------val_aucc:0.7343664459895957------test_aucc:0.7711996630116295------
------epoch: 7/10------val_aucc:0.7387630541330382------test_aucc:0.7719952732978091------
------epoch: 8/10------val_aucc:0.7376575656599017------test_aucc:0.7684554773237533------
------epoch: 9/10------val_aucc:0.7412883475165682------test_aucc:0.7667677934624005------
------epoch: 10/10------val_aucc:0.7423403675286265------test_aucc:0.7637412063787674------
------pl finetune finished!------val_aucc:0.7444387639114133------test_aucc:0.77312708508

In [8]:
#Maximum Entropy Regularized Loss
lambda_1 = 0.1

max_val_aucc = 0
tau = 5
aucc_list = []
model_mer = slearner_criteo_uplift(12, 4).cuda()
model_mer.load_state_dict(torch.load('Model/model_before_finetune.pkl'))
optimizer = optim.Adam(model_mer.parameters(), lr=1e-5, weight_decay=1e-5)
for epoch in range(num_epoch):
    count = 0
    for data in train_dl:
        optimizer.zero_grad()
        x, t, y_visit, y_visit_float, y_conv, y_conv_float = data
        x, t, y_visit, y_visit_float, y_conv, y_conv_float = x.cuda(), t.cuda(), y_visit.cuda(), y_visit_float.cuda(), y_conv.cuda(), y_conv_float.cuda()
        row_idx = torch.arange(x.shape[0])
        
        T1 = torch.ones_like(t).cuda()
        T0 = torch.zeros_like(t).cuda()
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_mer(x, T0, T1)
        t = t.squeeze()
        r = torch.cat([y_conv_pred_T0, y_conv_pred_T1], dim=1)
        c = torch.cat([y_visit_pred_T0, y_visit_pred_T1], dim=1)
        idx_T0 = torch.where(t==0)[0]
        idx_T1 = torch.where(t==1)[0]
        
        
        a1 = nn.functional.softmax((r-lambda_1*c)/tau, dim=1)

        loss_dfl = - torch.mean((y_conv - lambda_1*y_visit).squeeze().detach() * a1[row_idx, t.long()])

        loss = loss_dfl
        loss.backward()
        optimizer.step()
        count += 1

    with torch.no_grad():
        model_mer = model_mer.cpu()
        T1 = torch.ones_like(T_test)
        T0 = torch.zeros_like(T_test)
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_mer(X_test, T0, T1)
        conv_pred = y_conv_pred_T1 - y_conv_pred_T0
        visit_pred = y_visit_pred_T1 - y_visit_pred_T0
        conv_pred = conv_pred.numpy()
        visit_pred =  visit_pred.numpy()
        roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
        test_aucc = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
        model_mer = model_mer.cuda()

    with torch.no_grad():
        T1 = torch.ones_like(T_val).cuda()
        T0 = torch.zeros_like(T_val).cuda()
        y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_mer(X_val, T0, T1)
        conv_pred = y_conv_pred_T1 - y_conv_pred_T0
        visit_pred = y_visit_pred_T1 - y_visit_pred_T0
        conv_pred = conv_pred.cpu().numpy()
        visit_pred =  visit_pred.cpu().numpy()
        roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
        val_aucc = get_uplift_model_aucc(t=(T_val.cpu().numpy() > 0.5).flatten(), yr=Y_conv_val.cpu().numpy().flatten(), yc=Y_visit_val.cpu().numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
        if val_aucc[0] > max_val_aucc:
            max_val_aucc = val_aucc[0]
            torch.save(model_mer.state_dict(), 'Model/model_mer.pkl')

    print(f'------epoch: {epoch+1}/{num_epoch}------val_aucc:{val_aucc[0]}------test_aucc:{test_aucc[0]}------')



with torch.no_grad():
    model_mer_best = slearner_criteo_uplift(12, 4)
    model_mer_best.load_state_dict(torch.load('Model/model_mer.pkl'))
    T1 = torch.ones_like(T_test)
    T0 = torch.zeros_like(T_test)
    y_visit_pred_T0, y_visit_pred_T1, y_conv_pred_T0, y_conv_pred_T1 = model_mer_best(X_test, T0, T1)
    idx_T0 = torch.where(T_test==0)[0]
    idx_T1 = torch.where(T_test==1)[0]
    conv_pred = y_conv_pred_T1 - y_conv_pred_T0
    visit_pred = y_visit_pred_T1 - y_visit_pred_T0
    conv_pred = conv_pred.numpy()
    visit_pred =  visit_pred.numpy()
    roi_pred = conv_pred / np.where(abs(visit_pred) < 1e-6, 1e-6, visit_pred)
    test_aucc_mer = get_uplift_model_aucc(t=(T_test.numpy() > 0.5).flatten(), yr=Y_conv_test.numpy().flatten(), yc=Y_visit_test.numpy().flatten(), roi_pred=roi_pred.flatten(), quantile=200)
    print(f'------mer finetune finished!------val_aucc:{max_val_aucc}------test_aucc:{test_aucc_mer[0]}')


------epoch: 1/10------val_aucc:0.712076519788604------test_aucc:0.7688673565069325------
------epoch: 2/10------val_aucc:0.7129747429755668------test_aucc:0.7748737176264008------
------epoch: 3/10------val_aucc:0.71452045601545------test_aucc:0.7721834120693929------
------epoch: 4/10------val_aucc:0.7145159549427909------test_aucc:0.7742032103773447------
------epoch: 5/10------val_aucc:0.7259666608481063------test_aucc:0.7729711165156808------
------epoch: 6/10------val_aucc:0.7304515334250161------test_aucc:0.7739996278884348------
------epoch: 7/10------val_aucc:0.7292697607918609------test_aucc:0.7770062335607394------
------epoch: 8/10------val_aucc:0.7362007953686206------test_aucc:0.7791914249737131------
------epoch: 9/10------val_aucc:0.740817861401174------test_aucc:0.7793157537137336------
------epoch: 10/10------val_aucc:0.7410326410185315------test_aucc:0.7816443538969104------
------mer finetune finished!------val_aucc:0.7410326410185315------test_aucc:0.78164435389691

In [ ]:
plt.figure(figsize=(8, 5))
plt.rc('font', family='Times New Roman')

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

ax = plt.gca()
ax.spines['top'].set_visible(True)
ax.spines['top'].set_color('black')
ax.spines['top'].set_linewidth('1.5')

ax.spines['bottom'].set_visible(True)
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth('1.5')

ax.spines['left'].set_visible(True)
ax.spines['left'].set_color('black')
ax.spines['left'].set_linewidth('1.5')


ax.spines['right'].set_visible(True)
ax.spines['right'].set_color('black')
ax.spines['right'].set_linewidth('1.5')




x_index = test_aucc_ts[1]
y_random = test_aucc_ts[2][-1] / x_index[-1] * x_index

x_normalization = x_index[-1]
y_normalization = test_aucc_ts[2][-1]


plt.figure(1)
plt.plot(x_index / x_normalization, test_aucc_ts[2] / y_normalization, color='#BB9727', label='TSM-SL', linewidth=2)
plt.plot(x_index / x_normalization, test_aucc_ifd[2] / y_normalization, color='#F27970', label='DFCL-IFD', linewidth=2)
plt.plot(x_index / x_normalization, test_aucc_pl[2] / y_normalization, color='#05B9E2', label='DFCL-PL', linewidth=2)
plt.plot(x_index / x_normalization, test_aucc_mer[2] / y_normalization, color='#8983BF', label='DFCL-MER', linewidth=2)

plt.plot(x_index / x_normalization, y_random / y_normalization, color='#000000', label='Random', linewidth=2)

plt.xlabel('Incremental cost', fontsize=12, fontweight='bold')
plt.ylabel('Incremental reward', fontsize=12, fontweight='bold')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


plt.grid(True)
plt.legend(prop={'weight':"bold"})
plt.legend()

# plt.savefig('avg_aucc_criteo.pdf', format='pdf', bbox_inches = 'tight', dpi=1200)

plt.show()